# The General Simulator of Rankine Cycle 

* 1 The Representation of Rankine Cycle Flowsheet


* 2 The General Analysis of Rankine Cycle


## 1 The Representation of Rankine Cycle Flowsheet

**Step3** https://github.com/PySEE/PyRankine/tree/master/step3

**Basic Object-Orientation Abstraction**  and **Data Representation** of The Ideal Rankine Cycle(Example 8.1)

---
The importmant stage in the Rankine Cycle Simulator is to be able to generate a Rankine Cycle Flowsheet.

A Rankine Cycle Flowsheet should be able to describe the `nodes` and `components` present in a Rankine Cycle and also be able to describe how these components are `connected` to each other through `nodes`.

* 1 Rankine Cycle Representation as `.csv` files

* 2 Rankine Cycle Representation as `JSON` file

## 1.1  Rankine Cycle Representation as a Comma Separated Value (.csv) file

A `Comma Separated Value (.csv)` file can be opened and edited in any `spreadsheet` software like any other spreadsheet.

In the .csv file, each `row` of the spreadsheet appears in a separate `line` and the `contents of the cells` in a row are separated by `commas`.

A `.csv` file is essentially a `text` file which can be edited  by any Text Editor


### The  Rankine Cycle is `drawn` in two `.csv` files 

* **`nodes`** 

    The nodes would have `unique ID `(**NID**) for the nodes in the Rankine Cycle schematic


* **`components`:** `components` and how these components are `connected`  to each other through `nodes`.

   The components would have `unique symbols`(**TYPE**) for the components in the Rankine Cycle schematic

### Example: the CSV files of the Rankine Cycle 81 schematic

![rankine81](./img/rankine81.jpg)

CSV files of the Rankine Cycle 81 Flowsheet 

* `nodes`: rankine81-nds.csv
```
NAME,NID,p,t,x
Main Steam,0,8,,1
Outlet Steam of HP,1,0.008,,
Condenser Water,2,0.008,,0
Main FeedWater,3,8,,
```
* `components`: rankine81-dev.csv
```
NAME,TYPE,NODES
Turbine,TURBINE,0,1
Condenser,CONDENSER,1,2
Pump,PUMP,2,3
Boiler,BOILER,3,0
```

![rankine81-nds](./img/rankine81-nds.png)

![rankine81-nds-sp](./img/rankine81-nds-sp.png)

![rankine81-dev](./img/rankine81-dev.png)

![rankine81-dev-sp](./img/rankine81-dev-sp.png)


### The Cycle Simulator with cvs file

In [ ]:
"""
Step 3 ：Basic Object-Orientation Abstraction**  and Data Representation** of The Ideal Rankine Cycle(Example 8.1)

"""
import csv

import node
import turbine
import pump
import condenser
import boiler


def read_nodesfile(filename):
    """ nodes in the  csv file"""
    countNodes = len(open(filename, 'r').readlines()) - 1
    nodes = [None for i in range(countNodes)]
    csvfile = open(filename, 'r')
    reader = csv.DictReader(csvfile)
    for line in reader:
        i = int(line['NID'])
        nodes[i] = node.Node(line['NAME'], i)
        try:
            nodes[i].p = float(line['p'])
        except:
            nodes[i].p = None
        try:
            nodes[i].t = float(line['t'])
        except:
            nodes[i].t = None
        try:
            nodes[i].x = float(line['x'])
        except:
            nodes[i].x = None

        if line['p'] != '' and line['t'] != '':
            nodes[i].pt()
        elif line['p'] != '' and line['x'] != '':
            nodes[i].px()
        elif line['t'] != '' and line['x'] != '':
            nodes[i].tx()

    return nodes, countNodes


def read_devicefile(filename):
    devFile = open(filename, 'r')
    discardHeader = devFile.readline()
    Comps = {}
    i = 0
    begId = 2
    for line in devFile:
        dev = line.split(',')
        if dev[1] == "TURBINE":
            Comps[dev[0]] = turbine.Turbine(
                dev[0], int(dev[begId]),  int(dev[begId + 1]))
        elif dev[1] == "BOILER":
            Comps[dev[0]] = boiler.Boiler(
                dev[0], int(dev[begId]), int(dev[begId + 1]))
        elif dev[1] == "CONDENSER":
            Comps[dev[0]] = condenser.Condenser(
                dev[0], int(dev[begId]), int(dev[begId + 1]))
        elif dev[1] == "PUMP":
            Comps[dev[0]] = pump.Pump(dev[0], int(
                dev[begId]),  int(dev[begId + 1]))

        i = i + 1

    DevNum = i
    return Comps, DevNum


class RankineCycle(object):

    def __init__(self, name, nodefilename, devfilename):
        self.name = name
        self.nodes = []
        self.devs = {}
        self.nodes, self.NodeNum = read_nodesfile(nodefilename)
        self.devs, self.DevNum = read_devicefile(devfilename)

    def state(self):
        for key in self.devs:
            self.devs[key].state(self.nodes)

    def simulate(self):
        for key in self.devs:
            self.devs[key].simulate(self.nodes)

        self.bwr = self.devs['Pump'].workRequired / \
            self.devs['Turbine'].workExtracted
        self.efficiency = (self.devs['Turbine'].workExtracted - self.devs[
                           'Pump'].workRequired) / (self.devs['Boiler'].heatAdded)

    def spower_simulate(self, Wcycledot):
        self.Wcycledot = Wcycledot
        self.mdot = Wcycledot * 1000.0 * 3600.0 / \
            (self.devs['Turbine'].workExtracted -
             self.devs['Pump'].workRequired)
        for key in self.devs:
            self.devs[key].mdotenergy(self.mdot)

    def cw_simulate(self):
        """ Circulating water system：Condenser Cooling Water"""
        self.nodew = []
        self.nodew.append(node.Node('CW-Inlet', 0))
        self.nodew.append(node.Node('CW-Outlet', 1))

        self.nodew[0].t = 15
        self.nodew[0].x = 0
        self.nodew[1].t = 35
        self.nodew[1].x = 0
        self.nodew[0].tx()
        self.nodew[1].tx()

        self.devs['Condenser'].cw_nodes(0, 1)
        self.devs['Condenser'].cw_simulate(self.nodew)

    def export(self):
        print(" \n --------  %s   ----------------------------------" % self.name)
        print("The net power output: ", self.Wcycledot, "MW")
        print("Efficiency: ", '%.2f' % (self.efficiency * 100), "%")
        print("The back work ratio: ", '%.2f' % (self.bwr * 100), "%")
        print("The mass flow rate: ", '%.2f' % self.mdot, "kg/h")
        print('The rate of heat transfer as the fluid passes the boiler: ',
              '%.2f' % self.devs['Boiler'].Qindot, 'MW')
        print(" \n -------  Circulating Water System  --------------")
        print("Cooling water enters the condenser T:", self.nodew[0].t, u'°C')
        print("Cooling water exits  the condenser T:", self.nodew[1].t, u'°C')
        print('The rate of heat transfer from the condensing steam: ',
              '%.2f' % self.devs['Condenser'].Qoutdot, 'MW')
        print('The mass flow rate of the condenser cooling water: ', '%.2f' %
              self.devs['Condenser'].mcwdot, 'kg/h')
        print(" \n -------- NODES  -----------------------------------")
        print("\nNodeID\tName\tP\tT\tH\tS\tV\tX")
        for inode in self.nodes:
            print(inode)


if __name__ == '__main__':
    nds_filename = 'rankine81-nds.csv'
    dev_filename = 'rankine81-dev.csv'
    c81 = RankineCycle("Rankine81", nds_filename,  dev_filename)
    c81.state()
    c81.simulate()
    # Specified Net Output Power
    Wcycledot = 100
    c81.spower_simulate(Wcycledot)
    c81.cw_simulate()
    c81.export()

## 1.2 Rankine Cycle Representation as a JSON file

[JSON (JavaScript Object Notation)](http://json.org/), specified by [RFC 7159]() (which obsoletes RFC 4627) and by ECMA-404, is a `lightweight data interchange` format inspired by [JavaScript](https://en.wikipedia.org/wiki/JavaScript) object literal syntax (although it is not a strict subset of JavaScript).

### `JSON` is built on `two` structures:

* 1 A collection of `name/value` pairs. In various languages, this is realized as an `object`, `record`, `struct`, `dictionary`, `hash table`, `keyed list`, or `associative array`.


* 2 An **ordered** `list` of values. In most languages, this is realized as an `array`, `vector`, `list`, or `sequence`.

###  In `JSON`, they take on these `forms`

#### 1 An `object` is an `unordered` set of `name/value` pairs.

* An `object` begins with `{` (left brace) and ends with `}` (right brace).

* Each `name` is followed by `:` (colon) 

* The `name/value` pairs are `separated` by `, `(comma)

### 2 An `array` is an `ordered` collection of **values**

* An array begins with `[` (left bracket) and ends with `]` (right bracket).

* Values are separated by `,` (comma).

#### 3 A `value` can be a `string` in double quotes("), or a `number`, or `true` or `false` or `null`, or an `object` or an `array`

These structures can be **nested**.

#### 4 A **string** is a sequence of zero or more `Unicode` characters, wrapped in `double` quotes(`"`), using backslash escapes(`\`)

A `character` is represented as a single character `string`. 

A string is very much like a `C` or `Java` string.

#### 5 A `number` is very much like a `C` or `Java` number, `except` that the `octal and hexadecimal` formats are not used.

### The  Rankine Cycle is `drawn` in the json  files 

* `nodes`

  The nodes would have `unique ID `(**id**) for the nodes in the Rankine Cycle Flowsheet

* `components`

   The components would have `unique symbols`(**type**) for the components in the Rankine Cycle Flowsheet

### Example: the JSON representation of the Rankine Cycle 81　Flowsheet

![rankine81](./img/rankine81.jpg)

the json file of the Rankine Cycle 81 Flowsheet: **rankine81.json**

```json
{
    "name": "Rankine81",
    "nodes": [
        {
            "name": "Main Steam",
            "id": 0,
            "p": 8.0,
            "t": null,
            "x": 1,
            "fdot": 1
        },
        {
            "name": "Outlet Steam of HP",
            "id": 1,
            "p": 0.008,
            "t": null,
            "x": null,
            "fdot": null
        },
        {
            "name": "Condenser Water",
            "id": 2,
            "p": 0.008,
            "t": null,
            "x": 0,
            "fdot": null
        },
        {
            "name": "Main FeedWater",
            "id": 3,
            "p": 8.0,
            "t": null,
            "x": null,
            "fdot": null
        }
    ],
    "comps": [
        {
            "name": "Turbine",
            "type": "TURBINE-EX0",
            "eff": 100,
            "inNode": 0,
            "outNode": 1
        },
        {
            "name": "Condenser",
            "type": "CONDENSER",
            "inNode": 1,
            "outNode": 2
        },
        {
            "name": "Feedwater Pump",
            "type": "PUMP",
            "eff": 100,
            "inNode": 2,
            "outNode": 3
        },
        {
            "name": "Boiler",
            "type": "BOILER",
            "eff": null,
            "inNode": 3,
            "outNode": 0
        }
    ]
}
```

In [ ]:
"""
Step 3-json 

"""
import json

import node
import turbine
import pump
import condenser
import boiler


def read_jsonfile(filename):
    """ rankine cycle in json file"""

     # 1 read json file to dict 
    with open(filename, 'r') as f:
        rkcyc = json.loads(f.read())

    # print(rkcyc)   
    name=rkcyc["name"]
    dictnodes=rkcyc["nodes"]
    dictcomps=rkcyc["comps"]

    # 2 convert dict nodes to the object nodes
    countNodes=len(dictnodes)
    nodes = [None for i in range(countNodes)]
    for curjnode in  dictnodes:
        i = int(curjnode['id'])
        nodes[i] = node.Node(curjnode['name'], i)
        nodes[i].p = curjnode['p']
        nodes[i].t = curjnode['t']
        nodes[i].x = curjnode['x']
            
        if nodes[i].p!=None and nodes[i].t != None:
            nodes[i].pt()
        elif nodes[i].p!=None and nodes[i].x!=None:
            nodes[i].px()
        elif nodes[i].t!=None and nodes[i].x!=None:
            nodes[i].tx()
        
    #print(nodes[1]) 
      
    # 3 convert dict Comps to the object Comps
    DevNum=len(dictcomps)
    Comps = {}
    for curdev in dictcomps:
        if curdev['type'] == "TURBINE":
            Comps[curdev['name']] = turbine.Turbine(
                curdev['name'], curdev['inNode'],curdev['exNode'])
        elif curdev['type'] == "BOILER":
            Comps[curdev['name']] = boiler.Boiler(
                curdev['name'], curdev['inNode'],curdev['exNode'])
        elif curdev['type'] == "CONDENSER":
            Comps[curdev['name']] = condenser.Condenser(
                curdev['name'], curdev['inNode'],curdev['exNode'])
        elif curdev['type'] == "PUMP":
             Comps[curdev['name']] = pump.Pump(curdev['name'], curdev['inNode'],curdev['exNode'])
 
    return  name,nodes, countNodes,Comps, DevNum

class RankineCycle(object):

    def __init__(self, rankinefile):
        self.nodes = []
        self.devs = {}
        self.name,self.nodes, self.NodeNum,self.devs, self.DevNum= read_jsonfile(rankinefile)

    def state(self):
        for key in self.devs:
            self.devs[key].state(self.nodes)

    def simulate(self):
        for key in self.devs:
            self.devs[key].simulate(self.nodes)

        self.bwr = self.devs['Pump'].workRequired / \
            self.devs['Turbine'].workExtracted
        self.efficiency = (self.devs['Turbine'].workExtracted - self.devs[
                           'Pump'].workRequired) / (self.devs['Boiler'].heatAdded)

    def spower_simulate(self, Wcycledot):
        self.Wcycledot = Wcycledot
        self.mdot = Wcycledot * 1000.0 * 3600.0 / \
            (self.devs['Turbine'].workExtracted -
             self.devs['Pump'].workRequired)
        for key in self.devs:
            self.devs[key].mdotenergy(self.mdot)

    def cw_simulate(self):
        """ Circulating water system：Condenser Cooling Water"""
        self.nodew = []
        self.nodew.append(node.Node('CW-Inlet', 0))
        self.nodew.append(node.Node('CW-Outlet', 1))

        self.nodew[0].t = 15
        self.nodew[0].x = 0
        self.nodew[1].t = 35
        self.nodew[1].x = 0
        self.nodew[0].tx()
        self.nodew[1].tx()

        self.devs['Condenser'].cw_nodes(0, 1)
        self.devs['Condenser'].cw_simulate(self.nodew)

    def export(self):
        print(" \n --------  %s   ----------------------------------" % self.name)
        print("The net power output: ", self.Wcycledot, "MW")
        print("Efficiency: ", '%.2f' % (self.efficiency * 100), "%")
        print("The back work ratio: ", '%.2f' % (self.bwr * 100), "%")
        print("The mass flow rate: ", '%.2f' % self.mdot, "kg/h")
        print('The rate of heat transfer as the fluid passes the boiler: ',
              '%.2f' % self.devs['Boiler'].Qindot, 'MW')
        print(" \n -------  Circulating Water System  --------------")
        print("Cooling water enters the condenser T:", self.nodew[0].t, u'°C')
        print("Cooling water exits  the condenser T:", self.nodew[1].t, u'°C')
        print('The rate of heat transfer from the condensing steam: ',
              '%.2f' % self.devs['Condenser'].Qoutdot, 'MW')
        print('The mass flow rate of the condenser cooling water: ', '%.2f' %
              self.devs['Condenser'].mcwdot, 'kg/h')
        print(" \n -------- NODES  -----------------------------------")
        print("\nNodeID\tName\tP\tT\tH\tS\tV\tX")
        for inode in self.nodes:
            print(inode)


if __name__ == '__main__':
    rankine_filename = 'rankine81.json'
    c81 = RankineCycle(rankine_filename)
    c81.state()
    c81.simulate()
    # Specified Net Output Power
    Wcycledot = 100
    c81.spower_simulate(Wcycledot)
    c81.cw_simulate()
    c81.export()


## 2 The General Analysis of Rankine Cycle

[Step5: The Base Class of Components](https://github.com/PySEE/PyRankine/tree/master/step6)

The General Simulator of Rankine Cycle with the base class of components

* `json` file for the representation of Rankine Cycle Flowsheet

### Examples of Rankine Cycle
 
Michael J . Moran. Fundamentals of Engineering Thermodynamics(7th Edition). John Wiley & Sons, Inc. 2011

Chapter 8 : Vapour Power Systems Example

* [EXAMPLE 8.1: Analyzing an Ideal Rankine Cycle, P438](http://nbviewer.jupyter.org/github/PySEE/PyRankine/blob/master/notebook/RankineCycle81-82-Step0-1.ipynb)

* [EXAMPLE 8.5: The Regenerative Cycle with Open Feedwater Heater, P456](http://nbviewer.jupyter.org/github/PySEE/PyRankine/blob/master/notebook/RankineCycle85-Step0.ipynb)
 
 ![step5](./img/step5.jpg)


### The UML  Class Diagrams of the General Rankine Cycle Simulator   

[UML](https://en.wikipedia.org/wiki/Unified_Modeling_Language) is a conventional diagrammatic notation used to describe `class structures` and `other higher level aspects of software design`

####  [The Class diagram](https://en.wikipedia.org/wiki/Class_diagram)

The **class diagram** in  UML is a type of static structure diagram that describes the structure of a system by showing the system's classes, their attributes, operations (or methods), and the relationships among objects. 

A class is depicted on the class diagram as a rectangle with **three** horizontal sections:

* The `upper` section shows the class's `name`; 

* The`middle` section contains the class's `attributes`; 

* The `lower` section contains the class's `operations (or "methods")`. 

##### Relationships
 
UML relations notation the relationship is a general term covering the specific types of logical **connections** found on class and object diagrams. 

In the Class Diagram of Rankine Cycle,we use the following relationships: 

**1 Association(关联-Instance-level relationships)** 

Association represents the static relationship shared among the objects of two classes. 

The directional relationship represented by **a line with an arrowhead**. The arrowhead depicts a container-contained directional flow

**2 Composition(组合-Instance-level relationships)**

The UML representation of a composition relationship shows composition as **a filled diamond shape** on the containing class end of the lines that connect contained class(es) to the containing class.

**3 Inheritance(继承-Class-level relationships)** : 

To model inheritance on a class diagram, **a solid line** is drawn from the child class (the class inheriting the behavior) with **a closed, unfilled arrowhead (or triangle)** pointing to the super class

#### The Class Diagram：Association 

* 1 `rankine_cycle.py`:the General Simulator of Rankine Cycle

* 2 `rankine.py` : the main runner of the General Simulator of Rankine Cycle

![Package UML](./uml/step5-package.jpg)

#### The  Class Diagram : Inheritance, Composition
 
![Class UML](./uml/step5-classes.jpg)

#### The Rankine Cycle Classes 

##### 1 The `node` Class  

`component/node.py`

In [ ]:

"""
Step5: The General Simulator of Rankine Cycle with the  base class of components

    class Node
                      ──┐           ┌──
                        │           │
       component A      ├─⇒ Node ⇒─┤ component B
                        │           │
                      ──┘           └──

json object example:

    {
            "name": "Extracted Steam To Opened FWH",
            "id": 1,
            "p": 0.7,
            "t": null,
            "x": null,
            "fdot":null
    }
  
Author:Cheng Maohua  Email: cmh@seu.edu.cn  
  
"""
import seuif97 as if97


class Node(object):
  
    title = ('{:^6} \t {:^30} \t {:^5}\t {:^7}\t {:^7}\t {:^5} \t {:^7}\t {:^7}\t {:^5}\t {:^10}'.format
                 ("NodeID", "Name", "P", "T", "H", "S", "V", "X", "FDOT", "MDOT"))

    def __init__(self, name, nodeid):
        self.name = name
        self.nodeid = nodeid

        self.p = None
        self.t = None
        self.h = None
        self.s = None
        self.v = None
        self.x = None

        self.fdot = None
        self.mdot = None

    def calmdot(self, totalmass):
        self.mdot = totalmass * self.fdot

    def pt(self):
        self.h = if97.pt2h(self.p, self.t)
        self.s = if97.pt2s(self.p, self.t)
        self.v = if97.pt2v(self.p, self.t)
        self.x = if97.pt2x(self.p, self.t)

    def ph(self):
        self.t = if97.ph2t(self.p, self.h)
        self.s = if97.ph2s(self.p, self.h)
        self.v = if97.ph2v(self.p, self.h)
        self.x = if97.ph2x(self.p, self.h)

    def ps(self):
        self.t = if97.ps2t(self.p, self.s)
        self.h = if97.ps2h(self.p, self.s)
        self.v = if97.ps2v(self.p, self.s)
        self.x = if97.ps2x(self.p, self.s)

    def hs(self):
        self.t = if97.hs2t(self.h, self.s)
        self.p = if97.hs2p(self.h, self.s)
        self.v = if97.hs2v(self.h, self.s)
        self.x = if97.hs2x(self.h, self.s)

    def px(self):
        self.t = if97.px2t(self.p, self.x)
        self.h = if97.px2h(self.p, self.x)
        self.s = if97.px2s(self.p, self.x)
        self.v = if97.px2v(self.p, self.x)

    def tx(self):
        self.p = if97.tx2p(self.t, self.x)
        self.h = if97.tx2h(self.t, self.x)
        self.s = if97.tx2s(self.t, self.x)
        self.v = if97.tx2v(self.t, self.x)

    def __str__(self):
        result = ('{:^6d} \t {:<30} \t {:>6.3f}\t {:>7.3f}\t {:>7.2f}\t {:>5.2f} \t {:>7.3f}\t {:>5.3}\t {:>6.4f}\t {:>.2f}'.format
                  (self.nodeid, self.name, self.p, self.t, self.h, self.s, self.v, self.x, self.fdot, self.mdot))
        return result

#### 2 The base class of components

All component classes have the methods:

`__init__`,`state`,`fdot`,` simulate`, `sm_energy`,`export`

 `fdot`: check and analysis get **mass float rate** with mass and energy balance

* `_fdotok_`: the helper of fdot
 
The base class: `component/BComponent.py`

Two kinds of methods in the  base class

* 1 The methods must be implemented by its subclass

```python
def state(self, nodes):
    """ State """
    raise NotImplementedError
```

* 2 The methods can be inherited and overloaded by its subclass

```python
def export(self, nodes):
    """ export results: name,nodes """
    result = '\n' + self.name
    result += '\n' + Node.title
    for i in self.nodes:
       result +='\n' + nodes[i].__str__()
    return result
```

In [ ]:
from .node import Node


class BComponent():

    energy = None
    devTYPE = None

    def __init__(self, dictDev):
        """ Initializes the Component"""
        self.name = None
        self.nodes = []
        #self.__dict__.update(dictDev)
        
        self.fdotok = False

    def state(self, nodes):
        """  State """
        raise NotImplementedError

    # _fdotok_
    def _fdotok_(self, nodes):
        """ _fdotok_ """
        self.fdotok = nodes[self.nodes[0]].fdot != None
        for node in range(1, len(self.nodes)):
            self.fdotok = self.fdotok and (nodes[node].fdot != None)

    def fdot(self, nodes):
        """ mass and energy balance: innode->[]->outnode"""
        if (self.fdotok == False):
            try:
                if (nodes[self.inNode].fdot != None):
                    nodes[self.outNode].fdot = nodes[self.inNode].fdot
                elif (nodes[self.outNode].fdot != None):
                    nodes[self.inNode].fdot = nodes[self.outNode].fdot

                # modified self.fdotok
                self._fdotok_(nodes)
            except:
                self.fdotok == False

    def simulate(self, nodes):
        """  Simulates   """
        raise NotImplementedError

    def sm_energy(self, nodes):
        """  energy """
        raise NotImplementedError

    def export(self, nodes):
        """ export results: name,nodes """
        result = '\n' + self.name
        result += '\n' + Node.title
        for i in self.nodes:
           result +='\n' + nodes[i].__str__()
        return result


#### 3 The component class

The component classes are the subclass  of  the base class

* Boiler,TurbineEx0,TurbineEx1,OpenedheaterDw0,Pump,Condenser

![rankine86](./img/rankine85.jpg)

For example: the Boiler class: `component/BComponent.py`

```python
from .node import *
from .BComponent import BComponent


class Boiler(BComponent):
    
```

**Python’s super()**

* A subclass for extending a method from its parent class:

The `super()` works with no arguments.

This is convenient that you don't have to `hard-code the name of the base class` into every method that uses its parent methods.
        
```python
def export(self, nodes):
    # result=BComponent.export(self,nodes)
    result=super().export(nodes)
    result += '\nheatAdded(kJ/kg) \t%.2f \nQAdded(MW) \t%.2f' % (
    self.heatAdded, self.QAdded)
    return  result
```
        
```python
def fdot(self, nodes):
    # BComponent.fdot(self,nodes)
    super().fdot(nodes)
```

In [ ]:
from .node import *
from .BComponent import BComponent


class Boiler(BComponent):

    energy = "heatAdded"
    devTYPE = "BOILER"

    def __init__(self, dictDev):
        """
        Initializes the boiler
        """
        super().__init__(dictDev)
        
        self.name = dictDev['name']
        self.inNode = dictDev['inNode']
        self.outNode = dictDev['outNode']
        self.type = dictDev['type']
        
        # add nodes
        self.nodes = [self.inNode, self.outNode]
  
    def state(self, nodes):
        pass

    def fdot(self, nodes):
        super().fdot(nodes)

    def simulate(self, nodes):
        self.heatAdded = nodes[self.inNode].fdot * \
            (nodes[self.outNode].h - nodes[self.inNode].h)

    def sm_energy(self, nodes):
        self.QAdded = nodes[self.inNode].mdot * \
            (nodes[self.outNode].h - nodes[self.inNode].h)
        self.QAdded /= (3600.0 * 1000.0)

    def export(self, nodes):
        result=super().export(nodes)
        result += '\nheatAdded(kJ/kg) \t%.2f \nQAdded(MW) \t%.2f' % (
            self.heatAdded, self.QAdded)
        return  result


## Reference

* Unified Modeling Language https://en.wikipedia.org/wiki/Unified_Modeling_Language

* UML http://www.uml.org/


* Donald Bell. [UML basics:An introduction to the Unified Modeling Language](https://www.ibm.com/developerworks/rational/library/769.html)

   * [UML basics: The class diagram](https://www.ibm.com/developerworks/rational/library/content/RationalEdge/sep04/bell/index.html?ca=drs-)
   
   
* [Pyreverse: UML Diagrams for Python](https://www.logilab.org/blogentry/6883)

  * [Pylint](https://www.pylint.org/) is shipped with `Pyreverse` which creates UML diagrams for python code

* [Graphviz - Graph Visualization Software](http://www.graphviz.org/)